In [ ]:
!pip install datasets
! pip install evaluate
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, RobertaForMultipleChoice
import torch
import random
import os
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

# Roberta-base
Instead of directly trying to guess the word, our first attemp is to generate three other choices from the original dataset, in addtion to the answers, and then we will fine tune Roberta-base to choose the right answer out of the four answers. We just want to prove that the model can capture some information from the input sentences.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# sam's path
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/'

os.chdir(drive_path)

In [ ]:
def load_dataset_from_disk():
    '''
    This function is talored to this file, because we will call the following code under certain conditions.
    Also depend on how Elchana saves his dataset, the way we are loading dataset will change.
    The output of this function is always the same: a huggingface 'datasets' object.
    '''
    # Load data: the data is preporcessed, so we already added three more choices to the dataset
    modified_train_fp = 'Baselines/MultipleChoices/ProcessedDatasets/cryptonite-train-choice.jsonl'
    modified_val_fp = 'Baselines/MultipleChoices/ProcessedDatasets/cryptonite-val-choice.jsonl'
    modified_test_fp = 'Baselines/MultipleChoices/ProcessedDatasets/cryptonite-test-choice.jsonl'
    datasets = load_dataset('json', data_files={'train': modified_train_fp, 'validation': modified_val_fp, 'test': modified_test_fp})
    return datasets




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

def tokenize_function(sample):
    prompt = sample['clue']
    choice0 = sample['answer']
    choice1 = sample['choice1']
    choice2 = sample['choice2']
    choice3 = sample['choice3']
    prompts = [prompt, prompt, prompt, prompt]
    choices = [choice0, choice1, choice2, choice3]
    # shuffle the choices
    random.shuffle(choices)
    correct_choice = choices.index(choice0)
    labels = torch.tensor(correct_choice)
    # Each instance of the prompt corresponds to a different choice. so [prompt, prompt], [choice0, choice1]
    model_input = tokenizer(prompts, choices,
                            return_tensors="pt",
                            padding='max_length', # the data will be padded by the data collector
                            max_length=128, truncation=True # dynamic padding later
                            )
    model_input = {k: v for k, v in model_input.items()}
    model_input['labels'] = labels
    return model_input

def load_or_create_tokenized_dataset(tokenized_dataset_fp, create=False):
    '''
    Load or create tokenized dataset
    Notice: if there are new input data, and we want to recreate the tokenized
    dataset, we need to delete what's under tokenized_dataset_fp and run this function again.
    '''
    if create:
        # delete the directory tokenized_dataset_fp
        if os.path.exists(tokenized_dataset_fp):
            os.system('rm -rf ' + tokenized_dataset_fp)
    if os.path.exists(tokenized_dataset_fp):
        # load from disk
        tokenized_datasets = load_from_disk(tokenized_dataset_fp)
    else:
        # create
        datasets = load_dataset_from_disk()
        tokenized_datasets = datasets.map(tokenize_function)
        tokenized_datasets = tokenized_datasets.remove_columns(['publisher','date', 'author', 'number', 'orientation', 'clue', 'answer','enumeration', 'quick','sub_publisher', 'choice1','choice2','choice3'])
        tokenized_datasets.save_to_disk(tokenized_dataset_fp)
    return tokenized_datasets


tokenized_dataset_fp = 'Baselines/MultipleChoices/ProcessedDatasets/TokenizedDatasets/'
tokenized_datasets = load_or_create_tokenized_dataset(tokenized_dataset_fp)
tokenized_datasets.set_format("torch")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

*2024.07.06*
I met a problem, the output of the model is always in shape (batch_size * 4, 1), meaning, every multiple choice of four, the model outputs the possibility for each of the choices, and combine the result of the whole batch together. So the solution is the shape of input should be (batch_size, num_choices, representation_of_each_choice). I unsqueeze it too much. This works becasue I looked into the code, and they use shape(1) as number of choices.
Also the target should be (batch_size), meaning, 1 dimentional array.
This works for a batch, but doesn't work for a single input, so we need to unsqueeze everything for single input. {k: v.unsqueeze(0) for k, v in sample.items()}.

In [ ]:
# load model
model = RobertaForMultipleChoice.from_pretrained("FacebookAI/roberta-base")

# define the compute metrics
metric = evaluate.load("accuracy",)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# define the trainer arguments - hyper paramters
lr = 1e-5
batch_size = 32
num_epochs = 1
output_dir = d 'Baselines/MultipleChoices/Models/roberta-base'
eval_strategy = 'steps'
warmup_ratio = 0.1
fp16 = True
optim = 'adamw_hf'


hyperparameters = {
    'learning_rate': lr,
    'num_train_epochs': num_epochs,
    'per_device_train_batch_size': batch_size,
    'lr_scheduler_type': 'linear',
    'warmup_ratio': warmup_ratio,
    'fp16': fp16,
    'optim': optim,
    'output_dir': output_dir,
    'do_eval': True,
    'do_train': True
}


training_args = TrainingArguments(**hyperparameters)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# train
train_result = trainer.train()
metrics = train_result.metrics
metrics

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.237100
1000,0.685000
1500,0.557900
2000,0.490100
2500,0.453000
3000,0.430200
3500,0.416000
4000,0.405500
4500,0.374100
5000,0.379100


{'train_runtime': 11328.2827,
 'train_samples_per_second': 41.56,
 'train_steps_per_second': 1.299,
 'total_flos': 1.2387262509540557e+17,
 'train_loss': 0.391222359605368,
 'epoch': 1.0}